In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
new_df = df.sample(30000)

In [4]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [5]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
134844,How did Donald Trump win despite projections t...,How and Why did Donald Trump win the election?
48998,Do all current day African Americans trace the...,Were Jews black people?
166078,What are the most interesting tech startups in...,What are the best startups in New York?
324831,How should I get my startup funded?,How can I get funding for my startups?
64429,How is transportation planning in India?,What is the scope of transportation planning i...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [9]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [10]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
134844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166078,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
324831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
255697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
257308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [12]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
134844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166078,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
324831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
64429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
255697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
257308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7506666666666667

In [15]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7316666666666667